In [ ]:
class TimeoutError(Exception):
    pass

def handler(signum, frame):
    raise TimeoutError()

import signal
signal.signal(signal.SIGALRM, handler)

In [ ]:
e = 257
while True:
    fc = [99] + [randint(1,95) for _ in range(2)] +[0]
    print(fc)
    f = e^fc[0] - e^fc[1] + e^fc[2] - e^fc[3]
    try:
        signal.alarm(3)
        factors = []
        ff = f
        while True:
            k, _ = ecm.find_factor(ff)
            if k not in Primes():
                k = list(factor(k))[0][0]
            factors.append(k)
            ff = ff // k
            print(factors)
            if ff in Primes():
                factors.append(ff)
                break
    except:
        continue
    finally:
        signal.alarm(int(0))
    if len(factors) > 20 and factors[-1] > 2^128:
        break

In [ ]:
ck=(factors[-1]*factors[0])
primes = {}
while True:
    s = ck
    for z in sample(factors[1:-1],randint(1,len(factors)-2)):
        s *= z
    if s+1 in Primes():
        primes[(s+1).bit_length()] = s+1
        if 1024-(s+1).bit_length() in primes:
            break

In [1]:
p=158627236620300768971181186557083051904700510000783692409072828117061848752046688497054945636976504112400020292865851970160214447096142508572737633716464941029
q=852589032346331877254556899369819689521084661432329137739347490081593272245835945593546876840132632332627098317923374586919725141114954947862916016317
lfp = 22833087297707435274476653319694535358668459345931805419833394253100805072069926214209006712042748083175078800550748182539992298383467189932887
e = 257
N = p*q

k = 1024 - 96 # unknown size: 96 bits

x = polygen(ZZ)
f = x^99 - x^59 + x^48 - 1
relation_coeffs = [(99,1),(59,-1),(48,1),(0,-1)]

In [2]:
phi=(p-1)*(q-1)//gcd(p-1,q-1)

In [3]:
d = 1/Zmod(phi)(e)

In [4]:
import mvcoppersmith

def RSAPRG(s0, N, e, k):
    state = s0
    for i in range(50000):
        state = Integer(pow(state, e, N))
        yield (state % 2**k)

def attempt(diylll=False, mult=1, extras=[], useModN=True, outputs = None):
    #global prg,full,outputs,fullstates,which_outputs,bs,fss,ys,F,M,unknown_parts,scalefactors,ms,ML,I,IB,polys
    N = p*q
    if useModN:
        modn = N
    else:
        modn = p
    if not outputs:
        s0 = randint(1,N-1)
        print("Generating PRG outputs")
        prg = RSAPRG(s0=Integer(s0), N=N, e=e, k=k)
        full = RSAPRG(s0=Integer(s0), N=N, e=e, k=int(log(N,2)+1))
        howfar = max(a for (a,_) in relation_coeffs)+1
        print("number of prng: ",howfar)
        outputs = [next(prg) for _ in range(howfar)]
        fullstates = [next(full) for _ in range(howfar)]
        print("Outputs generated")

    print("---attack starts here---")
    which_outputs = [i for (i,c) in relation_coeffs]

    bs = [outputs[i] for i in which_outputs]
    #fss = [fullstates[i] for i in which_outputs] # for debugging
    #unknown_parts = [(fs-b)//(2**k) for (fs,b) in zip(fss,bs)] # for debugging
    #print(fss, unknown_parts)
    
    ys = polygens(ZZ,'y',len(bs))
    F = (Integer(pow(2,-(len(ys)//2)*k,modn)) * (
        prod(
            (2**k * ys[i] + bs[i])**(relation_coeffs[i][1])
            for i in range(len(relation_coeffs))
            if relation_coeffs[i][1] > 0
        )
        - prod(
            (2**k * ys[i] + bs[i])**(-relation_coeffs[i][1])
            for i in range(len(relation_coeffs))
            if relation_coeffs[i][1] < 0
        )
    )) % modn

    #assert F(*[(fs - b)//(2**k) for (b,fs) in zip(bs,fss)]) % modn == 0 # for debugging

    print("Making Coppersmith lattice...")
    mvcoppersmith.coppersmith_params(F, mult=mult, extras=extras)
    M, ms, scalefactors = mvcoppersmith.coppersmith_makelattice(F, N//(2**k), modn, mult=mult, extras=extras)
    print("detM = 2^", prod(M.diagonal()).nbits())

    #assert all(mvcoppersmith.vec_to_poly(vec,ms,scalefactors)(*unknown_parts) % modn**mult == 0 for vec in M) # for debugging

    assert not any(M[i].is_zero() for i in range(len(ms)))

    if diylll:
        filename = input("Output filename for lattice to be LLL-reduced: ")
        mvcoppersmith.export_lattice(M, filename)

        filename = input("Input filename for LLL-reduced lattice: ")
        ML = mvcoppersmith.import_lattice(filename)
    else:
        print("Running LLL on dimension %d lattice..." % len(M.rows()))
        ML = M.dense_matrix().LLL()
        print("Done running LLL.")

    global shortrows
    shortrows = [row for row in ML.rows() if row.norm(1) < modn**mult]
    print("Number of vectors that seem short enough: ", len(shortrows))
    if len(shortrows) == 0: return False
    polys = [mvcoppersmith.vec_to_poly(row, ms, scalefactors) for row in shortrows]
    #assert all( pol(*unknown_parts) == 0 for pol in polys)

    # polys now contains polynomials that evaluate to 0 over the integers at the solution.
    # The coefficients are huge (on the order of modn^mult), but the solutions we're looking for are (relatively) small.
    # We can dramatically speed up the groebner basis computation by working mod a prime instead of over ZZ.
    print("Finding prime to work mod (if slow, this can be precomputed)")
    grob_mod = next_prime(2 * N//(2**k)) # slow
    grob_ring = Zmod(grob_mod)
    I = ideal([f.change_ring(grob_ring) for f in polys])
    print("Taking Groebner basis")
    set_verbose(2)
    #IB = I.groebner_basis(algorithm='magma') # uncomment if magma is installed
    IB = I.groebner_basis() # comment out if magma is installed
    results = [(poly / poly.content()).univariate_polynomial() for poly in IB if poly.nvariables() == 1]
    if len(results) == 0:
        print("no results")
    for poly in results:
        print(poly)
        root = poly.roots(multiplicities=False)[0]
        if poly.variables()[0] == ys[0]:
            recovered_state = bs[0] + 2**k * (root.lift())
    return recovered_state

In [5]:
import pwn

In [6]:
re=pwn.remote('172.30.160.1',int(20230))
re.sendline('19:MEYCIQD2CdoDOWdvcr5lLPly8EfSt4yCtcDaKnimEZ10rb2MrAIhAL1cHabquOYNulSw8ldaaV+7SAp0i6/Ob+ZJ91ZiSm8s')
re.sendline(str(p))
re.sendline(str(q))
re.sendline(str(lfp))
re.sendline(str(lfp))
re.sendline(str(e))
re.sendline('1')
re.readuntil("[")
l=re.readuntil("]")
outputs = eval("["+l.decode())

[x] Opening connection to 172.30.160.1 on port 20230
[x] Opening connection to 172.30.160.1 on port 20230: Trying 172.30.160.1
[+] Opening connection to 172.30.160.1 on port 20230: Done


/tmp/ipykernel_808232/2635518492.py:2: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline('19:MEYCIQD2CdoDOWdvcr5lLPly8EfSt4yCtcDaKnimEZ10rb2MrAIhAL1cHabquOYNulSw8ldaaV+7SAp0i6/Ob+ZJ91ZiSm8s')
/tmp/ipykernel_808232/2635518492.py:3: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline(str(p))
/tmp/ipykernel_808232/2635518492.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline(str(q))
/tmp/ipykernel_808232/2635518492.py:5: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline(str(lfp))
/tmp/ipykernel_808232/2635518492.py:6: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline(str(lfp))
/tmp/ipykernel_808232/2635518492.py:7: BytesWarning: Text is not bytes; assuming ASCII, no 

In [7]:
state=attempt(outputs=outputs)

---attack starts here---
Making Coppersmith lattice...
determinant = p**6 B**8
1.03**49 B**8 N**6 < N**7
2**2.0896 B**8.00 < N
dimension 7
detM = 2^ 6907
Running LLL on dimension 7 lattice...
Done running LLL.
Number of vectors that seem short enough:  5
Finding prime to work mod (if slow, this can be precomputed)
Taking Groebner basis
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/general.lib (4.1.2.0,Feb_2019)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/ring.lib (4.3.1.2,Nov_2022)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/primdec.lib (4.2.1.1,Jul_2021)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/absfact.lib (4.1.2.0,Feb_2019)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/triang.lib (4.1.2.0,Feb_2019)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/random.lib (4.1.2.0,Feb_2019)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/elim.lib (4.1.2.0,Feb_2019)
// ** loaded /opt/Sagemath/bin/../share/singular/LIB/matrix.lib (4.1.2.0,Feb_2019)

In [8]:
re.sendline(str(state))

/tmp/ipykernel_808232/1282192166.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  re.sendline(str(state))


In [9]:
re.recvline()
re.recvline().decode()

'Predict PRNG state: flag{rS4_prИ9_¢٥บld_6e_B4¢kd00Я£D}\n'